In [9]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time

In [10]:
# Load YOLO model and classes
yolo = cv2.dnn.readNet("./yolov3-tiny.weights", "./yolov3-tiny.cfg")

classes = []
with open("./coco.names", "r") as f:
    classes = f.read().splitlines()

In [11]:
# Open video file
cap = cv2.VideoCapture("./video3.avi")

In [12]:
while True:
    # Read a frame from the video
    ret, frame = cap.read()

    if not ret:
        break  # Break the loop if the video ends

    height, width, _ = frame.shape

    # Perform object detection on the current frame
    blob = cv2.dnn.blobFromImage(frame, 1/255, (320, 320), (0, 0, 0), swapRB=True, crop=False)
    yolo.setInput(blob)
    output_layer_names = yolo.getUnconnectedOutLayersNames()
    layer_outputs = yolo.forward(output_layer_names)

    # Process the detections as before
    boxes = []
    confidences = []
    class_ids = []

    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Adjust the confidence threshold as needed
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold=0.5, nms_threshold=0.4)

    color_list = [(255, 0, 0)]
    num_classes = len(classes)

    font = cv2.FONT_HERSHEY_SIMPLEX
    colors = [color_list[i % len(color_list)] for i in range(num_classes)]
    
    # Check if indexes is not empty
    if isinstance(indexes, tuple):
        time.sleep(1)
        break  # Break the loop if the video ends

    for i in indexes.flatten():
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        confi = str(round(confidences[i], 2))
        color = colors[i]

        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, label + " " + confi, (x, y + 20), font, 1, (255, 255, 255), 2)
        
    # Display the resulting frame
    cv2.imshow("Object Detection", frame)

    # Exit if 'q' is pressed or if the video is over
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()
